In [1]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

/home/jy/anaconda3/envs/baichuan2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [3]:
model_id = "/data/usr/jy/Langchain-Chatchat/webui_pages/fine_tune/final_model/user_模型2401"
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id,trust_remote_code=True)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Loading checkpoint shards: 100%|██████████| 8/8 [00:10<00:00,  1.34s/it]
The model 'BaichuanForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'M

In [4]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=5000)
hf = HuggingFacePipeline(pipeline=pipe, callbacks=[StreamingStdOutCallbackHandler()])

The model 'BaichuanForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Reform

In [5]:
from langchain.prompts import PromptTemplate
template = """问题: {question}
请回答"""
prompt = PromptTemplate.from_template(template)
chain = prompt | hf
question = "模具是什么？"
print(chain.invoke({"question": question}))

：
模具是一种用于制造产品或零件的模具。它通常由金属或其他耐用材料制成，形状和尺寸与要制造的产品或零件相匹配。模具的主要功能是提供产品的模板，以便在压力、加热或其他工艺条件下成型或塑造材料。根据所需的最终产品，模具可以设计成简单的形状或复杂的结构。模具的使用可以大大提高生产效率和质量，降低成本，同时减少对原材料的浪费。


In [25]:
chunks = []
for chunk in chain.stream({"question": question}):
    chunks.append(chunk)
    print(chunk, end="|", flush=True)

：
模具是一种用于制造物品的工具，通常由金属或塑料制成。它具有预先设计好的形状和尺寸，用于将原材料（如金属、塑料或橡胶）塑造成所需的形状和尺寸。模具在制造业中发挥着重要作用，尤其是在汽车、电子产品、家具和建筑行业等领域。|

In [16]:
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
template = """You are a chatbot having a conversation with a human.
{chat_history}
Human: {human_input}
Chatbot:"""
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationSummaryBufferMemory(max_token_limit=10000)


ValidationError: 1 validation error for ConversationSummaryBufferMemory
llm
  field required (type=value_error.missing)

In [15]:
llm_chain = LLMChain(
    llm=hf,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
# 未完成
# 继续：https://python.langchain.com/docs/integrations/llms/huggingface_pipelines